In [2]:
import re
from itertools import groupby
from datetime import datetime, timedelta
from asgiref.sync import sync_to_async
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()
from PostAnalyzer.models import (
    Market,
    Symbol,
    PositionSide,
    MarginMode,
    EntryTarget
)

In [3]:
def returnValue(val):
    return val.group(1) if val else None

In [4]:
def isPredictMsg(msg):
    patterns = [
        r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT",
        r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)",
        r"Entry Targets:([\s\S]+?)Take-Profit",
        r"Market:\s*([A-Z]+)",
        r"(StopLoss|Description):\s*([\d.]|\w+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg, re.IGNORECASE) for pattern in patterns)

In [5]:
async def findPosition(msg):
    position_match = re.search(r"Position:\s*([A-Z]+)", msg)
    
    try:
        position_value = await sync_to_async(PositionSide.objects.get)(name=returnValue(position_match).upper())
        return position_value
    except:
        return None

In [6]:
async def findMarket(msg):
    market_match = re.search(r"Market:\s*([A-Z]+)", msg, re.IGNORECASE)
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=returnValue(market_match).upper())
        return market_value
    except:
        return None


In [7]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(Isolated|Cross)\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_type = await sync_to_async(MarginMode.objects.get)(name=returnValue(leverage_match).upper())   
        leverage_value = int(leverage_match.group(2).lower().replace("x",""))    
    else:
        leverage_type = None
        leverage_value = None
    
    return leverage_type, leverage_value

In [8]:
def findStopLoss(msg):
   return returnValue(re.search(r"StopLoss:\s*([\d.]+)", msg))

In [9]:
def findEntryTargets(msg):
    match = re.search(r"Entry Targets:([\s\S]+?)Take-Profit", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data))  if i % 2 == 1]
        

In [10]:
def findTakeProfits(msg):
    match = re.search(r"Take-Profit Targets:([\s\S]+?)(StopLoss|Description)", msg, re.IGNORECASE)
    if match:
        extracted_data = returnValue(match)
        return [float(x.strip()) for i, x in enumerate(re.findall(r"(\d+\.\d+|\d+)", extracted_data)) if i % 2 == 1]
        

In [16]:
async def findSymbol(msg):
    symbol_match = re.search(r"Symbol:\s*#?([A-Z0-9]+)[/\s]?USDT", msg, re.IGNORECASE)

    try:
        symbol_value = await sync_to_async(Symbol.objects.get)(asset=returnValue(symbol_match).upper())
        return symbol_value
    except:
        return None

In [19]:
msg = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #AVAX/USDT\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 3x\n\nEntry Targets: \n1) 11.80\n2) 11.502\n\nTake-Profit Targets: \n1) 12.025\n2) 12.281\n3) 12.564\n4) 12.950\n\nStopLoss: 11.19\nB.Z",    "edit_date": None,
    "media": None,
}

msg1 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #FILUSDT.P\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 4x\n\nEntry Targets: \n1) 5.430\n2) 5.212\n\nTake-Profit Targets: \n1) 5.517\n2) 5.608\n3) 5.715\n4) 5.823\n5) 5.982\n\nStopLoss: 5.154\nB.Z",    
    "edit_date": None,
    "media": None,
}

msg3 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #BTC/USDT Market: FUTURES Position: LONG Leverage: Isolated 6x Entry Targets: 1) 30000.54 2) 32000.43 3) 33000. Take-Profit Targets: 1) 30010.66 2) 30016 3) 30019 4) 30040 StopLoss: 29010 B.Z #test",    
    "edit_date": None,
    "media": None,
}

msg4 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #Galausdt Market: FUTURES Position: LONG Leverage: Isolated 6x Entry Targets: 1) 30000.54 2) 32000.43 3) 33000. Take-Profit Targets: 1) 30010.66 2) 30016 3) 30019 4) 30040 StopLoss: 29010 B.Z #test",    
    "edit_date": None,
    "media": None,
}

msg5 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #1000BONKUSDT.P\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 4x\n\nEntry Targets: \n1) 0.022713\n2) 0.021730\n\nTake-Profit Targets: \n1) 0.23111\n2) 0.23689\n3) 0.24366\n4) 0.25361\n\nStopLoss: 0.21400\nB.Z",    
    "edit_date": None,
    "media": None,
}

msg6 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #GTCUSDT.P\nMarket: FUTURES\nPosition: LONG\nLeverage: Isolated 4x\nOrder Type: Limit\n\nEntry Price: \n1) 1.139\n2) 1.096\n\nTake-Profit Targets: \n1) 1.155\n2) 1.178\n3) 1.205\n4) 1.234\n5) 1.271\n\nStopLoss: 1.079\nB.Z",    
    "edit_date": None,
    "media": None,
}

# spot
msg2 = {
    "id": 3084,
    "date": "2023-12-19T12:49:50+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #ETH/USDT\nMarket: SPOT\n\nEntry Targets: \n1) 2930.00\n2) 2870.00\n\nTake-Profit Targets: \n1) 3100.00\n2) 3200.00\n\n\nDescription: HOLD MODE\nمیان مدت\nB.Z",    
    "edit_date": None,
    "media": None,
}

string = msg6["message"]


# ************* FIND IS PREDICT *************
isPredict = isPredictMsg(string)
print(isPredict)


# ************* FIND SYMBOL *************
# method 1
# symbol_match = re.search(r"Symbol:(.+)", string)
# symbol_match = returnValue(symbol_match).split("USDT")[0].replace("/", "").replace(" ", "").replace("#", "")
 
# method 2
symbol = await findSymbol(string)
print(symbol)


# ************* FIND MARKET *************
market = await findMarket(string)
print(market.name)

if market.name != "SPOT":
    # ************* FIND POSITION *************
    position = await findPosition(string)
    print(position.name)

    # ************* FIND LEVERAGE *************
    marginMode, leverage = await findLeverage(string)
    print(marginMode.name, leverage)


# ************* FIND STOP LOSS *************
stopLoss = findStopLoss(string)
print(stopLoss)

# ************* FIND ENTRY TARGETS *************
entries = findEntryTargets(string)
print(entries)
 
# ************* FIND TAKE PROFITS *************
profits = findTakeProfits(string)
print(profits)

False
GTC-USDT
FUTURES
LONG
ISOLATED 4
1.079
None
[1.155, 1.178, 1.205, 1.234, 1.271]


In [13]:
# from bingx.api import BingxAPI
# from dotenv import dotenv_values
# from bingX.spot import Spot
# from bingX.perpetual.v2 import Perpetual

# # ****************************************************************************************************************************

# config = dotenv_values(".env")


# API_KEY = config["API_KEY"]
# SECRET_KEY = config["SECRET_KEY"]

# client = Spot(API_KEY, SECRET_KEY)
# client1 = Perpetual(API_KEY, SECRET_KEY)

# client.symbols()
# client1.trade_order()

ModuleNotFoundError: No module named 'bingX'

In [ ]:
from bingx.api import BingxAPI
from dotenv import dotenv_values

config = dotenv_values(".env")


API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")


entry_price = 2.6902  
stop_loss = 2.5460     
take_profit = 2.73  
symbol = "PHB-USDT" 
time_interval = "1m"
start_timestamp = 1717063704000

# 86400000 == milliseconds in a day
next_day = 1717063704000 + 86400000
1717063800000
# res = bingx.get_kline_data(symbol,time_interval ,1718442480000 ,1718528880000,1440)

reached_target = None
stop_loss_reached = None
entry_reached = None
take_profit_reached = None
finish = True
isEntrySuccess = False
i = 1

while finish:
    res = bingx.get_kline_data(symbol,time_interval ,start_timestamp ,next_day, 1440)
    for row in reversed(res):

        if not entry_reached and float(row['low']) <= entry_price:
            entry_reached = row
       
        if bool(entry_reached):
            if not bool(stop_loss_reached) and float(row['low']) <= stop_loss:
                if not reached_target:
                    reached_target = 'Stop-Loss'
                stop_loss_reached = row
            
            if not bool(take_profit_reached) and float(row['high']) >= take_profit:
                if not reached_target:
                    reached_target = 'Take-Profit'
                take_profit_reached = row
          

        if bool(take_profit_reached and stop_loss_reached):
            finish = False
            break
    start_timestamp = next_day
    next_day = start_timestamp + 86400000
    print(i)
    i += 1


if reached_target:
    print(f"The {reached_target} was reached first.")
else:
    print("Neither stop-loss nor take-profit was reached within the available data.")

print(take_profit_reached)
print(stop_loss_reached)
print(entry_reached)

# The Take-Profit was reached first.
# Take-Profit  {'open': '2.7291', 'close': '2.7361', 'high': '2.7361', 'low': '2.7244', 'volume': '19567.00', 'time': 1717155240000}
# Stop-Loss    {'open': '2.5846', 'close': '2.5376', 'high': '2.5949', 'low': '2.5333', 'volume': '14267.00', 'time': 1717783200000}
# Entry        {'open': '2.7033', 'close': '2.6899', 'high': '2.7033', 'low': '2.6848', 'volume': '53505.00', 'time': 1717132440000}




1
2
3
4
5
6
7
8
9
The Take-Profit was reached first.
{'open': '2.7291', 'close': '2.7361', 'high': '2.7361', 'low': '2.7244', 'volume': '19567.00', 'time': 1717155240000}
{'open': '2.5846', 'close': '2.5376', 'high': '2.5949', 'low': '2.5333', 'volume': '14267.00', 'time': 1717783200000}
{'open': '2.7033', 'close': '2.6899', 'high': '2.7033', 'low': '2.6848', 'volume': '53505.00', 'time': 1717132440000}
